In [1]:
import datetime
import numpy as np
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql import DataFrameStatFunctions as stat
import numpy as np
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, udf, when
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import Window

import requests
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from pyspark.sql import DataFrameStatFunctions as stat
from pyspark.sql import Window
import pyspark.sql.functions as F
import os
import time
import json
import pickle
import sys
import os

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

NIS_DATA_BASE_PATH = "gs://nis-segment-datasource-v3/processed/"
NIS_OLD_DATA_BASE_PATH = "gs://nis-localytics-datasource/processed/"

#conf = SparkConf().setAll([('spark.driver.memory', '20g'), ('spark.broadcast.blockSize', '10m'), ("spark.executor.instances", '30')])
#sc = SparkContext(conf=conf)
#sqlContext = SQLContext(sc)

conf = SparkConf().setAll([('spark.sql.broadcastTimeout',1000)
    # ('spark.driver.memory', '20g'), \
#                            ('spark.broadcast.blockSize', '10m'),\
#                            ('spark.dynamicAllocation.enabled','False'),\
#                            ('spark.executor.instances','9999')
# #                            ,("spark.sql.autoBroadcastJoinThreshold",-1)
                          ])
sc = SparkContext.getOrCreate(conf=conf)
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/29 11:59:14 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/29 11:59:14 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/29 11:59:14 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/29 11:59:14 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
sampled_dates = [datetime.datetime(2023,2,1,0,0,0), datetime.datetime(2023,2,6,0,0,0),datetime.datetime(2023,2,9,0,0,0),datetime.datetime(2023,2,14,0,0,0),
                datetime.datetime(2023,2,18,0,0,0),datetime.datetime(2023,2,20,0,0,0), datetime.datetime(2023,2,25,0,0,0)]

In [3]:
df_device_vec=sqlContext.read.parquet("gs://pvtrough_asia_south1/clustering/device_vectors_roberta_idf1_AVG_"+sampled_dates[0].strftime("%Y_%m_%d")+"/")

In [4]:
df_device_vec.show()

+--------------------+--------------------+
|            deviceId| topic_avg_embedding|
+--------------------+--------------------+
|001de295-f6f1-4d1...|[-0.0071697412530...|
|002b0703-52f7-4a4...|[-0.0194327286506...|
|007b6eeb-f375-4fb...|[-0.0250822885791...|
|0096d221-c9d8-4d1...|[-0.0065972892994...|
|0110a0c0-ccd5-42f...|[2.04012986000405...|
|01adacb0-6499-4b1...|[-0.0127121494026...|
|01b77caf-7210-45d...|[0.00915400042859...|
|02186794-1e6b-489...|[-0.0202706851736...|
|022df247-bfda-495...|[0.00127237560344...|
|023e9526-2818-4a3...|[-0.0132817400566...|
|02517492-6def-4b5...|[-0.0104616365139...|
|0253f6c2-257c-4ca...|[-0.0134072505927...|
|02868e40-cbd0-416...|[-0.0158373165248...|
|028e56e8-4bcf-482...|[-0.0093228422735...|
|02e86ee9-2531-4cc...|[-0.0149874750952...|
|03df70f1-e2f4-420...|[-9.5108791977620...|
|03f6224b-0c28-439...|[0.01132517481996...|
|0460e58f-70cf-48c...|[-0.0172295651232...|
|0478c826-da28-403...|[-0.0044839536797...|
|04ac7147-35d4-4e6...|[0.0062920

In [9]:
df_device_vec.count()

2795469

In [5]:
import pandas as pd


In [6]:
df_news_vec=pd.read_csv('gs://pvtrough_asia_south1/clustering/df_embedding_LM.csv')

In [7]:
df_news_vec.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,hid
0,l_ghte28qh,-0.057410,0.072684,0.097653,-0.090259,-0.343810,-0.105157,0.021728,0.063545,0.182640,...,-0.049555,-0.025578,0.013677,0.005283,0.090573,0.058347,-0.041583,0.066587,-0.033251,l_ghte28qh
1,l_tqfhaeaj,-0.046366,0.066981,0.050630,-0.143336,-0.141429,-0.039216,0.002915,0.044791,0.155185,...,-0.042353,-0.041155,-0.010604,-0.027780,0.012123,0.006033,-0.066333,0.072790,-0.030882,l_tqfhaeaj
2,ovjkzdst,-0.042388,0.045634,0.043625,-0.118311,-0.244444,-0.148261,-0.002687,-0.038602,0.184455,...,-0.132436,-0.018437,-0.021342,0.051693,0.021492,-0.040733,-0.152499,-0.017137,-0.067525,ovjkzdst
3,2kvkwz2g,-0.051170,0.022633,0.104455,-0.095581,0.021788,-0.027990,0.087394,0.016160,0.159274,...,-0.038707,-0.089911,0.030627,0.053641,0.092690,0.050080,-0.032340,0.078150,-0.013654,2kvkwz2g
4,28mgge2p,-0.021327,0.049713,0.039964,-0.030434,-0.035662,-0.142843,0.034750,-0.002756,0.217098,...,-0.049017,-0.076278,0.020809,0.068402,0.020633,0.120798,-0.142670,-0.003269,0.002409,28mgge2p


In [8]:
df_news_vec

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,hid
0,l_ghte28qh,-0.057410,0.072684,0.097653,-0.090259,-0.343810,-0.105157,0.021728,0.063545,0.182640,...,-0.049555,-0.025578,0.013677,0.005283,0.090573,0.058347,-0.041583,0.066587,-0.033251,l_ghte28qh
1,l_tqfhaeaj,-0.046366,0.066981,0.050630,-0.143336,-0.141429,-0.039216,0.002915,0.044791,0.155185,...,-0.042353,-0.041155,-0.010604,-0.027780,0.012123,0.006033,-0.066333,0.072790,-0.030882,l_tqfhaeaj
2,ovjkzdst,-0.042388,0.045634,0.043625,-0.118311,-0.244444,-0.148261,-0.002687,-0.038602,0.184455,...,-0.132436,-0.018437,-0.021342,0.051693,0.021492,-0.040733,-0.152499,-0.017137,-0.067525,ovjkzdst
3,2kvkwz2g,-0.051170,0.022633,0.104455,-0.095581,0.021788,-0.027990,0.087394,0.016160,0.159274,...,-0.038707,-0.089911,0.030627,0.053641,0.092690,0.050080,-0.032340,0.078150,-0.013654,2kvkwz2g
4,28mgge2p,-0.021327,0.049713,0.039964,-0.030434,-0.035662,-0.142843,0.034750,-0.002756,0.217098,...,-0.049017,-0.076278,0.020809,0.068402,0.020633,0.120798,-0.142670,-0.003269,0.002409,28mgge2p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42124,1gj2h6m9,-0.041069,0.042359,0.061673,-0.158995,-0.301543,0.024277,0.033576,0.035502,0.222680,...,-0.063656,-0.113372,0.026240,0.004484,0.047283,-0.087787,-0.180910,0.050370,0.006710,1gj2h6m9
42125,wpyx8hyr,-0.019182,-0.003988,0.038876,-0.159812,-0.166056,-0.114518,-0.024489,0.004984,0.190872,...,-0.068182,-0.076527,-0.018590,0.019739,0.043226,0.026624,-0.075802,0.017410,-0.030431,wpyx8hyr
42126,ealqjehc,-0.031036,0.067268,0.082610,-0.079843,-0.257179,-0.092372,0.034813,0.096044,0.255528,...,-0.027974,-0.056943,0.031356,0.033992,0.051898,0.140484,-0.104189,0.063808,-0.015181,ealqjehc
42127,l_wozpwwg8,-0.040607,0.058500,0.125291,-0.067588,-0.277310,-0.053339,0.039387,0.092011,0.179863,...,-0.106006,-0.026914,0.063314,-0.061714,0.047747,-0.040559,-0.042301,0.025674,-0.089704,l_wozpwwg8
